In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import warnings
import datetime
import math
import json

import pandas as pd
import geopandas as gpd
import numpy as np 
from tqdm.auto import tqdm

import src.data_utils as d


warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)


## Loading data

In [2]:
# If some files don't exist you should generate them via `notebooks/additional_data_preprocessing.ipynb`.

road_segments_path = '../data/road_segments_processed/road_segments_vds_uber.shp'
sample_submission_path = '../data/SampleSubmission.csv'
train_cleaned_path = '../data/train_cleaned.csv'
result_weather_path = '../data/weather_processed/weather.csv'
cameras_count_path = '../data/SANRAL_processed/cameras_counts.csv'
vds_locations_path = '../data/SANRAL_processed/vds_locations.csv'
result_hourly_path = '../data/SANRAL_processed/VDS_hourly_all.csv'
cameras_comm_dates_path = '../data/SANRAL_processed/cameras_comnissionning_dates.csv'
result_segment_ttime_path = '../data/uber_processed/segment_ttime_daily.csv'
sid_neigh_path = '../data/uber_processed/sid_neighbors.json'


In [ ]:
road_segments        = gpd.read_file(road_segments_path)

sid_neighbors        = json.load(open(), 'r')

sample_sub           = pd.read_csv(sample_submission_path)
train_raw            = pd.read_csv(train_cleaned_path, parse_dates=['Occurrence Local Date Time'])
weather              = pd.read_csv(result_weather_path, parse_dates=['datetime'])
road_segment_cam_cnt = pd.read_csv(cameras_count_path, sep=';')
VDS_locations        = pd.read_csv(vds_locations_path) 
VDS_hourly           = pd.read_csv(result_hourly_path, parse_dates=['Date of Collection Period'])
commission_dates     = pd.read_csv(cameras_comm_dates_path)
segment_ttime_daily  = pd.read_csv(result_segment_ttime_path)


## Preparing train data

In [ ]:
train_raw['longitude'] = pd.to_numeric(train_raw['longitude'], errors='coerce')
train_raw.dropna(subset=['longitude'], inplace=True)
train_raw['longitude'] = train_raw['longitude'].astype('float')
train_raw.head()

In [ ]:
sids = train_raw['road_segment_id'].unique()

dts  = pd.date_range('2016-01-01',
                     '2019-01-01 00:00:00',
                     freq="1h")

tr   = pd.DataFrame({'datetime':dts})

for sid in sids:
    tr[str(sid)] = 0
    events = train_raw.loc[train_raw['road_segment_id'] == sid]
    dts = events['Occurrence Local Date Time'].dt.floor('H')
    dates = dts.astype(str).unique()
    tr.loc[tr['datetime'].isin(dates), sid] = 1
tr.head()

In [ ]:
train = pd.DataFrame({
    'datetime x segment_id':np.concatenate([[str(x) + " x " + str(c) 
                                             for c in sids] 
                                            for x in tr['datetime']]),
    'datetime':np.concatenate([[str(x) for c in sids] for x in tr['datetime']]),
    'segment_id':np.concatenate([[str(c) for c in sids] for x in tr['datetime']]),
    'y':tr[sids].values.flatten()
})
train.head()

## Preparing Test Data


In [ ]:
sample_sub[['datetime','segment_id']] = sample_sub['datetime x segment_id'].str.split(' x ',expand=True)
test = sample_sub[['datetime x segment_id', 'datetime', 'segment_id', 'prediction']]
test = test.rename(columns={'prediction': 'y'})
test.head()

## Road Segments

In [ ]:
road_segments['LANES'].replace(0,1,inplace=True)
road_segments['lane_width']   = road_segments['WIDTH'] / road_segments['LANES']
road_segments['sinuosity']    = road_segments.geometry.apply(u.get_sinuosity) - 1
road_segments['lon_centroid'] = road_segments.geometry.apply(lambda s: s.centroid.coords[0][0])
road_segments['lat_centroid'] = road_segments.geometry.apply(lambda s: s.centroid.coords[0][1])
road_segments['dist_to_center'] = np.sqrt((road_segments.lon_centroid - 18.421148) ** 2 +
                                          (road_segments.lon_centroid + 33.920460) ** 2)
road_segments['orientation']  = road_segments['geometry'].apply(u.get_orientation)
road_segments.drop(['REGION','geometry'],axis=1,inplace=True)

#Adding camera count on each segment
road_segments = road_segments.merge(road_segment_cam_cnt, left_on='segment_id', right_on='segment_id', how='left')
road_segments[['camera_count','vms_count','vds_count']] = road_segments[['camera_count','vms_count','vds_count']].fillna(0)
road_segments['vds_id'] = road_segments['vds_id'].replace({'VDS139IO':'VDS139', 
                                                            'VDS912IO':'VDS912',
                                                            'VDS913IO':'VDS913',
                                                            'VDS914IO':'VDS914'}, regex=True)
road_segments.head()

In [ ]:
road_segments['num'] = road_segments['num'].astype(str)
road_segments['neighbors_to_center']   = road_segments['num'].astype(str)
road_segments['neighbors_from_center'] = road_segments['num'].astype(str)

for direction in sid_neighbors:
    for sid, neighbor in sid_neighbors[direction].items():
        neighbor = ','.join(str(x) for x in neighbor)
        road_segments['neighbors_{0}'.format(direction)].replace(sid, neighbor, inplace=True)
        
road_segments['neighbors_to_center'][road_segments['neighbors_to_center'] == road_segments['num']] = 'None'
road_segments['neighbors_from_center'][road_segments['neighbors_from_center'] == road_segments['num']] = 'None'

road_segments['neighbors'] = road_segments['neighbors_to_center'] + ',' + road_segments['neighbors_from_center']

is_not_none = lambda x: x != 'None'
road_segments['neighbors'] = road_segments['neighbors'].apply(lambda s: 
                                                              set(filter(is_not_none, list(str(s).split(',')))))

In [ ]:
num_to_neighbors = dict(zip(road_segments.num, road_segments['neighbors']))

In [ ]:
road_segments.drop(['neighbors', 'neighbors_to_center', 'neighbors_from_center'], axis=1, inplace=True)

In [ ]:
road_segments.head()

## VDS_data
 

In [ ]:
VDS_hourly = u.clean_vds_data(VDS_hourly)


dts = pd.date_range('2016-01-01', '2019-03-31 23:00:00', freq="1h")
tr = pd.DataFrame({'datetime':dts})
VDS_list = VDS_hourly.vds_id.unique()
VDS_hours = pd.DataFrame({
    'datetime': np.concatenate([[str(x) for c in VDS_list] for x in tr['datetime']]),
    'vds_id': np.concatenate([[str(c) for c in VDS_list] for x in tr['datetime']])}
)
VDS_hours['datetime'] = VDS_hours['datetime'].astype('datetime64[ns]')
VDS_hourly = VDS_hours.merge(VDS_hourly, how='left', on=['datetime', 'vds_id'])

In [ ]:
VDS_hourly['hour']    = VDS_hourly.datetime.dt.hour
VDS_hourly['weekday'] = VDS_hourly.datetime.dt.weekday
VDS_hourly['year']    = VDS_hourly.datetime.dt.year
VDS_hourly['month']   = VDS_hourly.datetime.dt.month
VDS_hourly['quarter'] = (VDS_hourly.datetime.dt.month - 1) // 3 + 1

VDS_hourly['traffic_unknown'] = VDS_hourly['traffic_total'].isnull().astype(int)
VDS_hourly['speed_unknown']   = VDS_hourly['avg_speed'].isnull().astype(int)

In [ ]:
cols_fill_na = ['traffic1', 'traffic2', 'traffic3', 'traffic_total', 'avg_speed']
for f in cols_fill_na:
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'year', 'month', 'weekday', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'month', 'weekday', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'weekday'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id'])[f].transform('mean'))

In [ ]:
VDS_hourly['mean_avg_speed'] = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['avg_speed'].transform('mean')
VDS_hourly['std_avg_speed']  = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['avg_speed'].transform('std')

VDS_hourly['mean_traffic']   = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['traffic_total'].transform('mean')
VDS_hourly['std_traffic']    = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['traffic_total'].transform('std')

VDS_hourly['rel_diff_avg_speed'] = (VDS_hourly['avg_speed'] - VDS_hourly['mean_avg_speed']) / VDS_hourly['mean_avg_speed']
VDS_hourly['rel_diff_traffic']   = (VDS_hourly['traffic_total'] - VDS_hourly['mean_traffic']) / VDS_hourly['mean_traffic']


In [ ]:
VDS_hourly = u.add_lag_features_vds(VDS_hourly)
VDS_hourly = VDS_hourly.drop(['hour', 'weekday', 'year', 'month', 'quarter'], axis=1)

VDS_hourly.head(2)

## Add lag features for uber traveltimes

In [ ]:
segment_ttime_daily = u.add_lag_features_uber(segment_ttime_daily)

In [ ]:
segment_ttime_daily.tail()

In [ ]:
segment_ttime_daily['weekday'] = segment_ttime_daily['date'].astype('datetime64[ns]').dt.weekday

In [ ]:
segment_ttime_daily['mean_average_ttime'] = segment_ttime_daily.groupby(['segment_id', 'weekday'])['average_ttime'].transform('mean')
segment_ttime_daily['std_average_ttime'] = segment_ttime_daily.groupby(['segment_id', 'weekday'])['average_ttime'].transform('std')

In [ ]:
segment_ttime_daily.drop('weekday', axis=1, inplace=True)

## Sun altitude and azimuth

In [ ]:


per = pd.date_range('2016-01-01', '2019-03-31 23:00:00', freq="1h")
sun_pos = pd.DataFrame({'datetime':per})

sun_pos['sun_alt'] = sun_pos['datetime'].apply(u.get_alt)
sun_pos['sun_az']  = sun_pos['datetime'].apply(u.get_az)

In [ ]:
train = train.loc[train['segment_id'].isin(test['segment_id'].tolist())]

train = u.add_time_features(train)
test  = u.add_time_features(test)

In [ ]:
#Mean accident coordinates for each segment
coords = train_raw.groupby('road_segment_id').mean()[['longitude', 'latitude']]

In [ ]:
weather['datetime']  = weather['datetime'].astype('datetime64[ns]')

def merge_dataframes(df: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(coords, left_on='segment_id', right_on='road_segment_id', how='left')
    df[['longitude', 'latitude']] = df[['longitude', 'latitude']].round(4)
    df = df.merge(road_segments, on='segment_id', how='left')
    df = df.merge(weather, on='datetime', how='left')
    df = df.merge(sun_pos, on='datetime', how='left')
    df = df.merge(VDS_hourly, on=['datetime', 'vds_id'], how='left')
    df = df.merge(segment_ttime_daily, on=['segment_id','date'], how='left')
    return df

train = merge_dataframes(train)
test  = merge_dataframes(test)

train = d.optimize(train)
test  = d.optimize(test)

## Calculating neighbor features for traveltime, speed and traffic

In [ ]:
for fval in ['traffic_total', 'avg_speed', 'average_ttime']:
    u.add_feature_based_on_neighbs(data=train, fval=fval, num_to_neighbors=num_to_neighbors)
    u.add_feature_based_on_neighbs(data=test, fval=fval, num_to_neighbors=num_to_neighbors)

In [ ]:
#Define day_period according to uber
b = [0,7,10,16,19,24]
l = ['AM', 'Early Morning','Midday','PM','Evening']
train['day_period'] = pd.cut(train['hour'], bins=b, labels=l, right=False, include_lowest=True)
test['day_period']  = pd.cut(train['hour'], bins=b, labels=l, right=False, include_lowest=True)

#Calculate angle between sun azimuth and road direction
train['blinding'] = abs(train['orientation'] - train['sun_az'])
test['blinding']  = abs(test['orientation'] - test['sun_az'])

#Calculate angle between wind and road directions
train['angle_wind_road'] = abs(train['orientation'] - train['wind_dir_angle'])
test['angle_wind_road']  = abs(test['orientation'] - test['wind_dir_angle'])

#Calculate distance from centroid to average accident location
train['dist_to_centroid'] = np.sqrt((train['lon_centroid'] - train['longitude']) ** 2 
                                    + (train['lat_centroid'] - train['latitude']) ** 2)

test['dist_to_centroid'] = np.sqrt((test['lon_centroid'] - test['longitude']) ** 2 
                                    + (test['lat_centroid'] - test['latitude']) ** 2)

train = u.add_public_holidays(train)
test  = u.add_public_holidays(test)
    
train = u.add_school_holidays(train)
test = u.add_school_holidays(test)

## Deleting cameras counts from records prior to commission_date

In [ ]:
commission_dates.groupby(['Commissioning Date','cls_segment_id'])['Asset Type'].agg('value_counts')

In [ ]:
train['vds_count'].loc[(train['segment_id'].isin(['5093ZHY','UY59I9M'])) & (train['datetime'] < '2016-02-05 07:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['EUXS47I','Q2V2552'])) & (train['datetime'] < '2016-07-05 12:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['YF1XDKK'])) & (train['datetime'] < '2016-07-05 12:00:00')] -= 2
train['camera_count'].loc[(train['segment_id'].isin(['812PNMZ'])) & (train['datetime'] < '2016-07-05 12:00:00')] -= 3
train['vds_count'].loc[(train['segment_id'].isin(['7QBQK9L','812PNMZ','F14EJLW'])) & (train['datetime'] < '2016-07-11 09:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['YF1XDKK'])) & (train['datetime'] < '2016-07-19 10:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['2M1UBKJ'])) & (train['datetime'] < '2016-12-08 08:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['X4UA382'])) & (train['datetime'] < '2016-12-08 09:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['N90YL69','OVJNYB2'])) & (train['datetime'] < '2017-02-02 07:00:00')] -= 1
train['vds_count'].loc[(train['segment_id'].isin(['RZK0UM0'])) & (train['datetime'] < '2017-02-15 11:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['N90YL69'])) & (train['datetime'] < '2017-03-20 07:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['B9V4IAJ'])) & (train['datetime'] < '2017-05-11 07:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['H9QJECU'])) & (train['datetime'] < '2017-10-24 10:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['MJ9YER2'])) & (train['datetime'] < '2017-11-04 07:00:00')] -= 1
train['vds_count'].loc[(train['segment_id'].isin(['8ITYQ2Z'])) & (train['datetime'] < '2017-11-07 07:00:00')] -= 1

## Removing silent periods

In [ ]:
train = u.proc_silent_intervals(train)

## Matching column names

## Adding last quarter count by sid-weekday-hour

In [ ]:
train['last_quarter'] = train['datetime'].dt.year + ((train['month'] - 1) // 3) * 0.25
test['last_quarter']  = test['datetime'].dt.year + ((test['month'] - 1) // 3) * 0.25

acc_cnt_quarter_sid_wdh = pd.DataFrame(train.groupby(['segment_id', 'last_quarter', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid_wd_h'})

acc_cnt_quarter_sid_wd = pd.DataFrame(train.groupby(['segment_id', 'last_quarter', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid_wd'})

acc_cnt_quarter_sid_h = pd.DataFrame(train.groupby(['segment_id', 'last_quarter', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid_h'})

acc_cnt_quarter_sid = pd.DataFrame(train.groupby(['segment_id', 'last_quarter'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid'})


acc_cnt_quarter_vds_wdh = pd.DataFrame(train.groupby(['vds_id', 'last_quarter', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds_wd_h'})

acc_cnt_quarter_vds_wd = pd.DataFrame(train.groupby(['vds_id', 'last_quarter', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds_wd'})

acc_cnt_quarter_vds_h = pd.DataFrame(train.groupby(['vds_id', 'last_quarter', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds_h'})

acc_cnt_quarter_vds = pd.DataFrame(train.groupby(['vds_id', 'last_quarter'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds'})



def add_last_quarter_counts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(acc_cnt_quarter_sid_wdh, on=['segment_id', 'last_quarter', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_sid_wd, on=['segment_id', 'last_quarter', 'weekday'], how='left')
    df = df.merge(acc_cnt_quarter_sid_h, on=['segment_id', 'last_quarter', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_sid, on=['segment_id', 'last_quarter'], how='left')
    df = df.merge(acc_cnt_quarter_vds_wdh, on=['vds_id', 'last_quarter', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_vds_wd, on=['vds_id', 'last_quarter', 'weekday'], how='left')
    df = df.merge(acc_cnt_quarter_vds_h, on=['vds_id', 'last_quarter', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_vds, on=['vds_id', 'last_quarter'], how='left')

    return df

train['last_quarter'] -= 0.25
test['last_quarter']  -= 0.25

train = add_last_quarter_counts(train)
test  = add_last_quarter_counts(test)

In [ ]:
train = d.optimize(train)
test = d.optimize(test)

## Adding last halfyear count by sid-weekday-hour

In [ ]:
train['last_halfyear'] = train['datetime'].dt.year + ((train['month'] - 1) // 6) * 0.5
test['last_halfyear']  = test['datetime'].dt.year + ((test['month'] - 1) // 6) * 0.5

acc_cnt_halfyear_sid_wdh = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid_wd_h'})

acc_cnt_halfyear_sid_wd = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid_wd'})

acc_cnt_halfyear_sid_h = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid_h'})

acc_cnt_halfyear_sid = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid'})


acc_cnt_halfyear_vds_wdh = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds_wd_h'})

acc_cnt_halfyear_vds_wd = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds_wd'})

acc_cnt_halfyear_vds_h = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds_h'})

acc_cnt_halfyear_vds = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds'})



def add_last_halfyear_counts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(acc_cnt_halfyear_sid_wdh, on=['segment_id', 'last_halfyear', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_sid_wd, on=['segment_id', 'last_halfyear', 'weekday'], how='left')
    df = df.merge(acc_cnt_halfyear_sid_h, on=['segment_id', 'last_halfyear', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_sid, on=['segment_id', 'last_halfyear'], how='left')
    df = df.merge(acc_cnt_halfyear_vds_wdh, on=['vds_id', 'last_halfyear', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_vds_wd, on=['vds_id', 'last_halfyear', 'weekday'], how='left')
    df = df.merge(acc_cnt_halfyear_vds_h, on=['vds_id', 'last_halfyear', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_vds, on=['vds_id', 'last_halfyear'], how='left')

    return df

train['last_halfyear'] -= 0.5
test['last_halfyear']  -= 0.5

train = add_last_halfyear_counts(train)
test  = add_last_halfyear_counts(test)

In [ ]:
train = d.optimize(train)
test = d.optimize(test)

## Adding neighbor features for sids and last quarter/ halfyear

In [ ]:
for fval in ['acc_cnt_last_quarter_sid_wd_h', 'acc_cnt_last_quarter_sid_wd',
            'acc_cnt_last_quarter_sid_h','acc_cnt_last_quarter_sid',
             
            'acc_cnt_last_halfyear_sid_wd_h', 'acc_cnt_last_halfyear_sid_wd',
            'acc_cnt_last_halfyear_sid_h', 'acc_cnt_last_halfyear_sid']:
    
    add_feature_based_on_neighbs(data=train, fval=fval, num_to_neighbors=num_to_neighbors)
    add_feature_based_on_neighbs(data=test, fval=fval, num_to_neighbors=num_to_neighbors)

In [ ]:
train = d.optimize(train)
test = d.optimize(test)

In [ ]:
train.drop(['date', 'num', 'last_quarter', 'last_halfyear'], axis=1).to_pickle('../data/train_1002.pkl')
test.drop(['date', 'num', 'last_quarter', 'last_halfyear'], axis=1).to_pickle('../data/test_1002.pkl')